In [1143]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [1144]:
class Neural_Network_Image:
    def __init__(self):
        #For Training
        self.hidden_neurons = 15
        self.alpha = 0.1
        self.iter = 500
        self.test_ratio = 0.2
        self.x_test, self.y_test = None, None
        self.x_train, self.y_train = None, None
        self.input_size = None
        self.output_size = None
        ###############
        self.n_train = None #Sample size
        self.W1, self.b1, self.W2, self.b2 = None, None, None, None
    
    def prepare_data(self, data):
        data = np.array(data)
        m, _ = data.shape
        self.input_size = 784
        self.output_size = 10
        np.random.shuffle(data)
        
        test_size = int(self.test_ratio * m)
        test_set = data[:test_size].T
        self.y_test = test_set[0]
        self.x_test = test_set[1:] / 255.0

        self.n_train = m - test_size
        train_set = data[test_size:].T
        self.y_train = train_set[0]
        self.x_train = train_set[1:] / 255.0

    def prepare_data_iris(self, data):
        data = np.array(data)
        np.random.shuffle(data)
        self.input_size = 4
        self.output_size = 3
        np.random.shuffle(data)
        m, _ = data.shape
        test_size = int(self.test_ratio * m)
        test_set = data[:test_size].T
        self.y_test = test_set[-1].astype(int)
        self.x_test = test_set[:-1]
        
        self.n_train = m - test_size
        train_set = data[test_size:].T
        self.y_train = train_set[-1].astype(int)
        self.x_train = train_set[:-1]

    def prepare_data_tensorflow(self, type):
        #This is only for MNIST and FASHION of Tensorflow's database
        if type=='fashion':
            fashion_mnist = tf.keras.datasets.fashion_mnist
            (x_train, self.y_train), (x_test, self.y_test) = fashion_mnist.load_data()
        if type=='mnist':
            mnist = tf.keras.datasets.mnist
            (x_train, self.y_train), (x_test, self.y_test) = mnist.load_data()
        self.x_train = x_train.T.reshape(784,60000) / 255.0
        self.x_test = x_test.T.reshape(784,10000) / 255.0
        self.input_size = 784
        self.output_size = 10
        self.n_train = 60000
    
    def init_params(self):
        W1 = np.random.rand(self.hidden_neurons, self.input_size) - 0.5
        b1 = np.random.rand(self.hidden_neurons, 1) - 0.5
        W2 = np.random.rand(self.output_size, self.hidden_neurons) - 0.5
        b2 = np.random.rand(self.output_size, 1) - 0.5
        return W1, b1, W2, b2
    
    def ReLU(self, Z):
        return np.maximum(0, Z)
  
    def ReLU_deriv(self, Z):
        return Z > 0

    def SoftMax(self, Z):
        A = np.exp(Z) / sum(np.exp(Z))
        return A

    def one_hot(self, Y):
        one_hot_Y = np.zeros((Y.size, Y.max() + 1)) #For digit reg, 10 x m
        one_hot_Y[np.arange(Y.size), Y] = 1
        return one_hot_Y.T
    
    def get_predictions(self, A2):
        return np.argmax(A2, 0)

    def get_accuracy(self, predictions, actual):
        passed = np.sum(predictions == actual)
        accuracy = passed / actual.size
        return passed, accuracy
    
    def forward(self, X, W1, b1, W2, b2):
        Z1 = W1 @ X
        A1 = self.ReLU(Z1 + b1)
        Z2 = W2 @ A1
        A2 = self.SoftMax(Z2 + b2)
        return Z1, A1, A2
    
    def backward(self, X, one_hot_Y, Z1, A1, A2, W2):
        dZ2 =  A2 - one_hot_Y
        dW2 =  1/self.n_train * dZ2 @ A1.T
        db2 = 1/self.n_train * np.sum(dZ2)
        dZ1 = W2.T @ dZ2 * self.ReLU_deriv(Z1)
        dW1 = 1/self.n_train * dZ1 @ X.T
        db1 = 1/self.n_train * np.sum(dZ1)
        return dW1, db1, dW2, db2
    
    def gradient_descent(self):
        one_hot_Y = self.one_hot(self.y_train)
        W1, b1, W2, b2 = self.init_params()
        for i in range(self.iter+1):
            Z1, A1, A2 = self.forward(self.x_train, W1, b1, W2, b2)
            dW1, db1, dW2, db2 = self.backward(self.x_train, one_hot_Y, Z1, A1, A2, W2)
            if (i % 100 == 0):
                predictions = self.get_predictions(A2)
                passed, accuracy = self.get_accuracy(predictions, self.y_train)
                print(f'epoch {i}_passed tests {passed}/{self.n_train} || accuracy {(accuracy * 100):.3f} %')
        
            #Update Weights and biases
            W1 = W1 - self.alpha * dW1
            b1 = b1 - self.alpha * db1
            W2 = W2 - self.alpha * dW2
            b2 = b2 - self.alpha * db2
        
        self.W1, self.b1, self.W2, self.b2 = W1, b1, W2, b2

    def predict(self, test_data=None, test_label=None):
        if test_data == None:
            test_data = self.x_test
            test_label = self.y_test
        _,_,A2 = self.forward(test_data, self.W1, self.b1, self.W2, self.b2)
        predictions = self.get_predictions(A2)
        passed ,accuracy = self.get_accuracy(predictions, test_label)
        print(f'passed tests {passed}/{test_label.size} || accuracy {(accuracy * 100):.2f} %')

    def fit(self, train_data=None, tensorflow_data=None):
        if tensorflow_data == 'mnist':
            #Options: mnist or fashion
            self.prepare_data_tensorflow(tensorflow_data)
        elif tensorflow_data == 'fashion':
            self.prepare_data_iris(train_data)
        else:
            self.prepare_data(train_data)
        self.gradient_descent()

In [1145]:
data = pd.read_csv("/Users/tanhoangminhco/Documents/Coding/Python/Machine Learning/datasets/mnist_test.csv")
# iris = pd.read_csv('/Users/tanhoangminhco/Documents/Coding/Python/Machine Learning/datasets/iris2.csv')

In [1146]:
nn = Neural_Network_Image()
nn.fit(train_data=data, tensorflow_data=None)

epoch 0_passed tests 959/8000 || accuracy 11.987 %
epoch 100_passed tests 5902/8000 || accuracy 73.775 %
epoch 200_passed tests 6466/8000 || accuracy 80.825 %
epoch 300_passed tests 6701/8000 || accuracy 83.762 %
epoch 400_passed tests 6878/8000 || accuracy 85.975 %
epoch 500_passed tests 7011/8000 || accuracy 87.638 %


In [1147]:
nn.predict()

passed tests 1727/2000 || accuracy 86.35 %
